In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
%gui qt

In [5]:
from spiketag.base import *

In [6]:
from spiketag.view import *

In [7]:
prb = probe(shank_no=1)
prb[0] = np.array([0,1,2])

In [8]:
prb.mapping[0] = np.array([-10,0])
prb.mapping[1] = np.array([10,0])
prb.mapping[2] = np.array([0,10])

In [9]:
prb.fs = 20000.
prb.n_ch = 3
prb.reorder_by_chip=False

In [11]:
mua = MUA(mua_filename='./cell_0109.bin', spk_filename='./cell_0109.spk.bin',probe=prb, binary_radix=11, scale=False,
          cutoff=[-100,100], time_segs=np.array([[0, 40], [100., 200.]]) )

2019-03-13 15:20:37,243 - spiketag - INFO - #############  load data  ###################
2019-03-13 15:20:37,244 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2019-03-13 15:20:37,245 - spiketag - INFO - 9280040.0 * 3 points (111360480 bytes) 
2019-03-13 15:20:37,245 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2019-03-13 15:20:37,246 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2019-03-13 15:20:37,247 - spiketag - INFO - #############################################
2019-03-13 15:20:37,247 - spiketag - INFO - processing folder: ./
2019-03-13 15:20:37,254 - spiketag - INFO - raw data have 9309 spks
2019-03-13 15:20:37,255 - spiketag - INFO - ----------------success------------------
2019-03-13 15:20:37,255 - spiketag - INFO -  


In [12]:
mua.pivotal_pos

array([[   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
       [      1,       1,       1, ...,       1,       1,       1]],
      dtype=int32)

In [13]:
mua.pivotal_pos[0]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [14]:
spks = np.fromfile('./cell_0109.spk.bin', dtype=np.int32)

In [15]:
spks[::2]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [16]:
np.allclose(mua.pivotal_pos[0], spks[::2])

True

## Unit test for individual views

In [17]:
wview = wave_view(mua.data, spks=spks)
wview.show()

In [18]:
spk = mua.tospk()

2019-03-13 15:21:23,346 - spiketag - INFO - mua.tospk() with time_cutoff=True, amp_cutoff=True, speed_cutoff=False
2019-03-13 15:21:23,486 - spiketag - INFO - group 0 delete 100.0%(2884.0/2884.0) spks via cutoff
2019-03-13 15:21:23,488 - spiketag - INFO - ----------------success------------------
2019-03-13 15:21:23,488 - spiketag - INFO -  


In [19]:
mua.spk_times[0]/mua.fs

array([], dtype=float64)

In [20]:
mua.t

array([0.0000000e+00, 5.0000000e-05, 1.0000000e-04, ..., 4.6400185e+02,
       4.6400190e+02, 4.6400195e+02])

In [21]:
time_segs = np.array([[0, 40], [100., 200.]])

In [22]:
mua.spk_times[0][np.logical_and(mua.spk_times[0]/mua.fs<time_segs[1][1], mua.spk_times[0]/mua.fs>time_segs[1][0])].shape

(0,)

In [23]:
mua.spk_times[0].shape

(0,)

In [22]:
def find_spk_in_time_seg(spk_times, time_segs):
    spk_times_in_range = []
    for time_seg in time_segs:
        spk_in_time_seg = spk_times[logical_and(spk_times<time_seg[1], spk_times>time_seg[0])]
        spk_times_in_range.append(spk_in_time_seg)
    return np.hstack(np.array(spk_times_in_range))

In [24]:
%%timeit 
spk_times = find_spk_in_time_seg(mua.spk_times[0]/mua.fs, time_segs)

31.7 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
spk.spk[0].shape

(2838, 19, 3)

In [24]:
s = np.delete(spk.spk[0], [0,1,2], axis=0)

In [25]:
s.shape

(2835, 19, 3)

In [24]:
fet = spk.tofet()

2019-03-13 15:21:43,361 - spiketag - INFO - spk._tofet(group_id=0, method=pca, ncomp=6, whiten=False)
2019-03-13 15:21:43,362 - spiketag - INFO - ----------------success------------------
2019-03-13 15:21:43,363 - spiketag - INFO -  


In [30]:
clu = fet.toclu(method='dpgmm', group_id=0, n_comp=8, max_iter=400)

KeyError: 0

In [26]:
spkview = spike_view()

In [27]:
spkview.set_data(spk.spk[0], clu[0])

NameError: name 'clu' is not defined

In [35]:
spkview.show()

ERROR: Invoking <bound method BaseCanvas.on_draw of <spike_view (PyQt5) at 0x150027780>> repeat 2


In [31]:
fetview = scatter_3d_view()

In [32]:
fetview.set_data(fet.fet[0], clu[0])

In [33]:
fetview.show()

In [36]:
ampview = amplitude_view(fs=prb.fs, scale=1)

In [37]:
ampview.set_data(spk.spk[0], clu[0], mua.pivotal_pos[0])

In [38]:
ampview.show()

In [36]:
treeview = ctree_view()

In [37]:
treeview.set_data(clu[0])

In [38]:
treeview.show()

  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 427, in run_forever
    self._run_once()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 1440, in _run_once
    handle._run()
  File "/anaconda2/envs

In [39]:
corview = correlogram_view(fs=prb.fs)

In [40]:
corview.set_data(clu[0], mua.pivotal_pos[0])

AssertionError: 

In [44]:
corview.show()

In [45]:
traceview = trace_view(fs=prb.fs, spklen=19)

In [46]:
traceview.set_data(mua.data, clu[0], mua.pivotal_pos[0])

In [47]:
traceview.show()

## Unit test for QT5 Wrapper

In [10]:
from spiketag.mvc import MainModel, MainView

/anaconda2/envs/py37/lib/python3.6/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [11]:
model = MainModel(mua_filename='./cell_0109.bin', spk_filename='./cell_0109.spk.bin', probe=prb, 
                  binary_radix=11, scale=False)

2019-03-13 15:32:42,482 - spiketag - INFO - load mua data
2019-03-13 15:32:42,553 - spiketag - INFO - #############  load data  ###################
2019-03-13 15:32:42,554 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2019-03-13 15:32:42,555 - spiketag - INFO - 9280040.0 * 3 points (111360480 bytes) 
2019-03-13 15:32:42,556 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2019-03-13 15:32:42,557 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2019-03-13 15:32:42,558 - spiketag - INFO - #############################################
2019-03-13 15:32:42,560 - spiketag - INFO - processing folder: ./
2019-03-13 15:32:42,565 - spiketag - INFO - raw data have 9309 spks
2019-03-13 15:32:42,566 - spiketag - INFO - ----------------success------------------
2019-03-13 15:32:42,567 - spiketag - INFO -  
2019-03-13 15:32:42,568 - spiketag - INFO - extract spikes from pivital meta data
2019-03-13 15:32:42,569 - spiketag - INFO - mua.tospk() with time_cutoff=True, am

In [13]:
model.sort(clu_method='dpgmm', group_id=0, n_comp=8, max_iter=400)

2019-03-13 15:32:56,637 - spiketag - INFO - clustering with dpgmm
2019-03-13 15:32:56,809 - spiketag - INFO - Model.spktag is generated, nspk:9309


<function cluster._dpgmm at 0x12ef85620>
{'n_comp': 8, 'max_iter': 400}
0 [2 4 1 ... 4 4 6]
0 cluster finished


In [14]:
view = MainView(prb)

AttributeError: 'MainView' object has no attribute '_model'

In [48]:
view.show()

ERROR: Invoking <bound method SceneCanvas.on_draw of <ctree_view (PyQt5) at 0x183217908>> repeat 4
ERROR: Invoking <bound method SceneCanvas.on_draw of <ctree_view (PyQt5) at 0x183217908>> repeat 8


In [46]:
view.set_data(0, model.mua, model.spk[0], model.fet[0], model.clu[0])

  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 427, in run_forever
    self._run_once()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 1440, in _run_once
    handle._run()
  File "/anaconda2/envs

In [2]:
model.clu[0].index_count

NameError: name 'model' is not defined

In [ ]:
model.clu[0].select(np.arange(600))

In [15]:
thr = -31.00

In [16]:
idx = np.where(model.spk[0].min(axis=1).min(axis=1)>thr)[0]

In [17]:
idx.shape

(5032,)

In [17]:
def delete_spk(model, spk_idx):
    model.mua.spk_times[0] = np.delete(model.mua.spk_times[0], spk_idx, axis=0)
    model.spk[0] = np.delete(model.spk[0], spk_idx, axis=0)
    model.fet[0] = model.spk._tofet(0, method='pca')
    model.cluster(group_id=0, method='hdbscan', fall_off_size=30) #, 
    view.set_data(0, model.mua, model.spk[0], model.fet[0], model.clu[0])

(4067,)
(2589,)
(2104,)


2018-06-07 16:28:50,182 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,532 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,760 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,946 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,118 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,289 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,453 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,632 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,843 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,147 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,437 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,671 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,932 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,445 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,680 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,938 - spiketag - DEBUG - no more undo
2018-06-07 16:28:54,136 - spiketag - DEBUG - no more undo


In [19]:
delete_spk(model, idx)

2018-06-07 16:16:50,296 - spiketag - DEBUG - ctree view expand clustier 4279 here


In [16]:
@view.ampview.clip.connect
def on_clip(thres):
    idx = np.where(model.spk[0].min(axis=1).min(axis=1)>thres)[0]
    print(idx.shape)
    delete_spk(model=model, spk_idx=idx)

  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 832, in start
    self._run_callback(self._callbacks.popleft())
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/anaconda2/lib/python2.7/site-package

(4817,)


In [42]:
model.spk[0].shape

(1678, 19, 3)

In [39]:
model.fet = model.spk.tofet()

2018-06-07 11:33:46,742 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-06-07 11:33:46,746 - spiketag - INFO - ----------------success------------------
2018-06-07 11:33:46,747 - spiketag - INFO -  


In [40]:
model.fet[0].shape

(1678, 6)

In [41]:
model.clu = model.fet.toclu()

2018-06-07 11:33:49,754 - spiketag - INFO - clustering finished, used 0.066253900528 sec


In [19]:
model.mua.spk_times[0].shape

(4740,)

In [16]:
view.ampview.poses[:, 1].min()

-146.53759765625

In [53]:
from spiketag.mvc.Control import controller

In [54]:
ctrl = controller(probe=prb, mua_filename='./cell_0109.bin', spk_filename='./cell_0109.spk.bin')

2018-12-28 20:20:37,338 - spiketag - INFO - load mua data
2018-12-28 20:20:37,401 - spiketag - INFO - #############  load data  ###################
2018-12-28 20:20:37,402 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-12-28 20:20:37,403 - spiketag - INFO - 9280040.0 * 3 points (111360480 bytes) 
2018-12-28 20:20:37,404 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2018-12-28 20:20:37,404 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2018-12-28 20:20:37,405 - spiketag - INFO - #############################################
2018-12-28 20:20:37,470 - spiketag - INFO - processing folder: ./
2018-12-28 20:20:37,473 - spiketag - INFO - raw data have 9309 spks
2018-12-28 20:20:37,474 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:37,475 - spiketag - INFO -  


In [55]:
ctrl.sort()

2018-12-28 20:20:46,070 - spiketag - INFO - extract spikes from pivital meta data
2018-12-28 20:20:46,071 - spiketag - INFO - mua.tospk()
2018-12-28 20:20:46,378 - spiketag - INFO - group 0 delete 0.0%(0.0/9309.0) spks via cutoff
2018-12-28 20:20:46,381 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:46,381 - spiketag - INFO -  
2018-12-28 20:20:46,382 - spiketag - INFO - grouping spike time
2018-12-28 20:20:46,382 - spiketag - INFO - extrat features with weighted-pca
2018-12-28 20:20:46,404 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-12-28 20:20:46,404 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:46,405 - spiketag - INFO -  
2018-12-28 20:20:46,406 - spiketag - INFO - clustering with hdbscan
2018-12-28 20:20:46,406 - spiketag - INFO - clustering start with 24 cpus
2018-12-28 20:20:50,116 - spiketag - INFO - clustering finished, used 3.7082719802856445 sec
2018-12-28 20:20:5

In [56]:
ctrl.show()

  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 427, in run_forever
    self._run_once()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 1440, in _run_once
    handle._run()
  File "/anaconda2/envs

array([-19.429688, -18.166504, -17.284668, ..., -17.5     , -19.528809,
       -28.05664 ], dtype=float32)

In [14]:
model.mua.spk_times[0].shape

(9309,)

In [15]:
model.spk[0].shape

(9309, 19, 3)

In [16]:
model.fet[0].shape

(9309, 6)

In [28]:
import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

class sorter(QWidget):

    def __init__(self):
        super(sorter, self).__init__()
        self.initUI()

    def initUI(self):

        hbox = QHBoxLayout(self)
        self.splitter1 = QSplitter(Qt.Horizontal)
#         textedit = QTextEdit()
#         self.splitter1.addWidget(self.topleft)
#         self.splitter1.addWidget(textedit)
#         self.splitter1.setSizes([100,200])
        self.splitter2 = QSplitter(Qt.Horizontal)
        self.splitter_fet = QSplitter(Qt.Vertical)

        self.splitter3 = QSplitter(Qt.Vertical)
        self.splitter3.addWidget(self.splitter1)
        self.splitter3.addWidget(self.splitter2)
#         self.splitter2.addWidget(self.bottom)

        hbox.addWidget(self.splitter3)

        self.setLayout(hbox)
        QApplication.setStyle(QStyleFactory.create('Cleanlooks'))

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('spiketag')
#         self.show()

    def set_data(self, mua, spk, fet, clu):
        ### init view and set_data
        self.spkview = spike_view()
        self.spkview.set_data(spk, clu)
        self.fetview0 = scatter_3d_view()
        self.fetview0.set_data(fet[:,[0,1,2]].copy(), clu)
        self.fetview1 = scatter_3d_view()
        self.fetview1.set_data(fet[:,[1,3,4]].copy(), clu)
        self.ampview = amplitude_view(fs=fs, scale=1)
        self.ampview.set_data(spk, clu, mua.pivotal_pos[0])
        self.treeview = ctree_view()
        self.treeview.set_data(clu)
        self.corview = correlogram_view(fs=fs)
        self.corview.set_data(clu, mua.pivotal_pos[0])
        self.traceview = trace_view(fs=mua.fs, spklen=spk.shape[1])
        self.traceview.set_data(mua.data, clu, mua.pivotal_pos[0])
#         self.traceview.locate_buffer = 2000
        
        ### put views into splitter
        self.splitter1.addWidget(self.traceview.native)
        self.splitter1.addWidget(self.splitter_fet)
        self.splitter_fet.addWidget(self.fetview0.native)
        self.splitter_fet.addWidget(self.fetview1.native)
        self.splitter1.addWidget(self.spkview.native)
        
        self.splitter2.addWidget(self.corview.native)
        self.splitter2.addWidget(self.treeview.native)
        self.splitter2.addWidget(self.ampview.native)


In [29]:
%gui qt

In [30]:
ex = sorter()

In [31]:
spk = mua.tospk()
fet = spk.tofet(method='pca', ncomp=9, whiten=False)
clu = fet.toclu(method='hdbscan')

2018-06-06 18:53:54,900 - spiketag - INFO - mua.tospk()
2018-06-06 18:53:54,909 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,910 - spiketag - INFO -  
2018-06-06 18:53:54,943 - spiketag - INFO - spk._tofet(groupNo=0, method=pca, ncomp=9, whiten=False)
2018-06-06 18:53:54,945 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,946 - spiketag - INFO -  
2018-06-06 18:53:55,855 - spiketag - INFO - clustering finished, used 0.906870126724 sec


In [32]:
ex.set_data(mua, spk.spk[1], fet.fet[1], clu[1])

KeyError: 1

In [66]:
ex.show()

In [70]:
ex.traceview.locate_buffer = 300

In [97]:
ex.fetview0.set_data(fet.fet[1][:,[1,3,4]], clu[1])
ex.update()

ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4774b690>> repeat 8192
ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4a0aa110>> repeat 8192


In [20]:
gclu = clu[1]

In [21]:
gclu.selectlist

array([ 848, 2681, 2795, 2899, 2951, 3162, 3521, 4018, 4031, 4074, 4169,
       4224, 4237, 4486, 4489, 4648, 5064, 5078, 5823, 5966, 5967, 6047,
       6049, 6070, 6610, 7207, 7319, 7919, 8057, 8167, 8378, 8602, 8679,
       9240])

2018-03-11 15:08:33,804 - spiketag - DEBUG - ctree view expand clustier 9311 here
2018-03-11 15:08:51,809 - spiketag - DEBUG - ctree view expand clustier 9313 here


In [21]:
gclu._registered_func_name('select')

['spiketag.view.spike_view.on_select_id121320795744',
 'spiketag.view.scatter_3d_view.on_select_id121444857928',
 'spiketag.view.scatter_3d_view.on_select_id121449105600',
 'spiketag.view.amplitude_view.on_select_id121452650320',
 'spiketag.view.trace_view.on_select_id121529220464']

2018-03-11 14:14:28,666 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:29,401 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:31,183 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:32,756 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,080 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,932 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:35,486 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,014 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,999 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:37,535 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:39,278 - spiketag - DEBUG - ctree view expand clustier 9312 here
2018-03-11 14:14:43,798 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-